In [1]:
from wrapper_functions import *

2022-10-23 19:15:34.339891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which is considered *experimental*."
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which i

TensorFlow version: 2.10.0. Expected: 2.7.0
TensorFlow Probability version: 0.18.0. Expected: 0.15.0


2022-10-23 19:16:05.426621: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Experiment II: 2D Spatial Field

### Model Configs

In [2]:
# Optimization configs. 
# Consider reduce below parameters / set to `False` if MCMC is taking too long:
# mcmc_num_steps, mcmc_burnin, mcmc_nchain, mcmc_initialize_from_map.
map_step_size=5e-4   # @param
map_num_steps=10_000  # @param

mcmc_step_size=1e-4 # @param
mcmc_num_steps=1000 # @param

mcmc_nchain=10 # @param
mcmc_burnin=2_500 # @param
bne_mcmc_initialize_from_map="True" # @param ["False", "True"]

bne_mcmc_initialize_from_map = eval(bne_mcmc_initialize_from_map)


In [3]:
# BMA parameters.
y_noise_std = 0.01  # Note: Changed from 0.1 # @param
bma_gp_lengthscale = .1 # @param
bma_gp_l2_regularizer = 0.5 # @param

bma_n_samples_train = 100 # @param
bma_n_samples_eval = 250 # @param
bma_n_samples_test = 250 # @param
bma_seed = 0 # @param


In [4]:
# BNE parameters.
bne_gp_lengthscale = .05 # 5. # @param
bne_gp_l2_regularizer = 1. # 15 # @param
bne_variance_prior_mean = -2.5 # @param
bne_skewness_prior_mean = -2.5 # @param
bne_seed = 0 # @param

### Read training/prediction data

In [5]:
training_eastMA = pd.read_csv('../data/training_dataset/training_eastMA.csv')
training_eastMA_noMI = training_eastMA[:51]
training_eastMA_folds = pd.read_csv('../data/training_dataset/training_eastMA_folds.csv')
base_model_predictions_eastMA = pd.read_csv('../data/prediction_dataset/base_model_predictions_eastMA.csv')
display(training_eastMA.shape, training_eastMA_folds.shape, base_model_predictions_eastMA.shape)
print("pred longitude max and min", base_model_predictions_eastMA["lon"].max(),base_model_predictions_eastMA["lon"].min())
print("pred latitude max and min", base_model_predictions_eastMA["lat"].max(),base_model_predictions_eastMA["lat"].min())
#list(base_model_predictions_eastMA.columns)
print("train longitude max and min", training_eastMA["lon"].max(),training_eastMA["lon"].min())
print("train latitude max and min", training_eastMA["lat"].max(),training_eastMA["lat"].min())


training51= pd.read_csv('../data/training_dataset/training51.csv')

(55, 7)

(55, 10)

(84421, 5)

pred longitude max and min -69.93 -73.5
pred latitude max and min 44.3 40.6
train longitude max and min -70.023598 -73.443056
train latitude max and min 44.107524 40.74529


In [6]:
# standardize
X_train1 = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
X_test1 = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
X_valid = np.concatenate((X_train1, X_test1), axis=0)
X_centr = np.mean(X_valid, axis=0)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

X_train1 = (X_train1 - X_centr) / X_scale
X_test1 = (X_test1 - X_centr) / X_scale

Y_train = np.expand_dims(training_eastMA_noMI["aqs"], 1).astype(np.float32)
#Y_test = np.expand_dims(base_model_predictions_eastMA["pred_av"], 1).astype(np.float32)

print("2011 center and scale: ", X_centr, X_scale)

2011 center and scale:  [-72.185104  42.680347] [3.5699997 3.7000008]


In [7]:
base_model_names = ["pred_av", "pred_gs", "pred_caces"]
base_preds_train = tf.stack([training_eastMA_noMI[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
base_preds_test = tf.stack([base_model_predictions_eastMA[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
#base_preds_test
display(base_preds_train.shape, base_preds_test.shape)

TensorShape([51, 3])

TensorShape([84421, 3])

## Bayesian Model Averaging

A Bayesian ensemble model where ensemble weights $w_k's$ are parameterized by Gaussian process priors:

$y \sim N(\mu(x), \sigma^2)$ 

$\mu(x) = \sum_{k=1}^K w_k(x) * m_k(x) \quad$  where $\{m_k\}_{k=1}^K$ are base model predictions.

$w(x) = softmax(f(x)) \qquad\;\;\;$ where $w=[w_1, \dots, w_K]$ and $f=[f_1, \dots, f_K]$

$f \stackrel{i.i.d.}{\sim} GaussianProcess(0, k)$




In [8]:
# Assemble into configs.
bma_model_config = DEFAULT_GP_CONFIG.copy()
map_config = DEFAULT_MAP_CONFIG.copy()
mcmc_config = DEFAULT_MCMC_CONFIG.copy()

bma_model_config.update(dict(lengthscale=bma_gp_lengthscale,
                             l2_regularizer=bma_gp_l2_regularizer,
                             y_noise_std=y_noise_std,
                             activation='relu'))

map_config.update(dict(learning_rate=map_step_size,
                       num_steps=map_num_steps))

mcmc_config.update(dict(step_size=mcmc_step_size, 
                        num_steps=mcmc_num_steps,
                       burnin=mcmc_burnin,
                       nchain=mcmc_nchain))

### Check 10-fold Random Cross Validation RMSE

In [50]:
ref_model = LinearRegression()
kf = KFold(n_splits=10, random_state=bma_seed, shuffle=True) 

rmse_lr = []
rmse_bma = []

for train_index, test_index in kf.split(X_train1):
      #print("Train:", train_index, "Validation:",test_index)
      X_tr, X_te = X_train1[train_index], X_train1[test_index] 
      Y_tr, Y_te = Y_train[train_index], Y_train[test_index]
      
      # Ref: linear regression
      ref_model.fit(X_tr, Y_tr)
      Y_pred = ref_model.predict(X_te)
      rmse_lr.append(rmse(Y_te, Y_pred))
      print(rmse_lr)

      base_preds_tr, base_preds_te = base_preds_train.numpy()[train_index], base_preds_train.numpy()[test_index]
      print(X_tr.shape, X_te.shape, Y_tr.shape, Y_te.shape, base_preds_tr.shape, base_preds_te.shape)

      # build model & run MCMC
      bma_prior, bma_gp_config = bma_dist(X_tr, 
                                    base_preds_tr, 
                                    **bma_model_config)

      bma_model_config.update(bma_gp_config)


      bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_tr, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


      bma_joint_samples = make_bma_samples(X_te, None, base_preds_te, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config,
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)
      y_pred = bma_joint_samples['y']
      y_pred = tf.reduce_mean(y_pred, axis=0)

      rmse_bma.append(rmse(Y_te, y_pred))
      print(rmse_bma)


[1.6682196]
(45, 2) (6, 2) (45, 1) (6, 1) (45, 3) (6, 3)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


KeyboardInterrupt: 

In [35]:
print("RMSE LR: ", np.mean(rmse_lr), np.std(rmse_lr))
print("RMSE BMA: ", np.mean(rmse_bma), np.std(rmse_bma))

RMSE LR:  1.6487318 0.5513062
RMSE BMA:  1.1578394 0.4248702


### Check 10-fold Spatial Cross Validation RMSE

In [9]:
# fold_id=10
# X_te = X_train1[training51.index[training51["fold"] == fold_id]]
# X_tr = X_train1[training51.index[training51["fold"] != fold_id]]
# Y_te = Y_train[training51.index[training51["fold"] == fold_id]]
# Y_tr = Y_train[training51.index[training51["fold"] != fold_id]]

# base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
# base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

# # build model & run MCMC
# bma_prior, bma_gp_config = bma_dist(X_tr, 
#                                     base_preds_tr, 
#                                     **bma_model_config)

# bma_model_config.update(bma_gp_config)


# bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
#                                            model_config=bma_model_config,
#                                            Y=Y_tr, 
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config)


# bma_joint_samples = make_bma_samples(X_te, None, base_preds_te, 
#                                      bma_weight_samples=bma_gp_w_samples[0],
#                                      bma_model_config=bma_model_config,
#                                      n_samples=bma_n_samples_eval, 
#                                      seed=bne_seed,
#                                      y_samples_only=False)
# y_pred = bma_joint_samples['y']
# y_pred = tf.reduce_mean(y_pred, axis=0)

# print(rmse(Y_te, y_pred))

# # Construct data from BMA samples, shapes (num_samples * num_data, ...)
# means_tr_mcmc, X_tr_mcmc, Y_tr_mcmc = make_bma_samples(
#     X_tr, Y_tr, base_preds_tr, 
#     bma_weight_samples=bma_gp_w_samples[0],
#     bma_model_config=bma_model_config,
#     n_samples=bma_n_samples_train,
#     seed=bma_seed, 
#     prepare_mcmc_training=True)

# # Mean samples based on test data, shape (num_samples, num_data, num_output).
# # It is used to generate final examples in `make_bne_samples()`.
# means_te_mcmc = make_bma_samples(
#     X_te, None, base_preds_te, 
#     bma_weight_samples=bma_gp_w_samples[0],
#     bma_model_config=bma_model_config,
#     n_samples=bma_n_samples_test,
#     seed=bma_seed)

In [10]:

# rmse_bma = []
# rmse_bne = []

# for fold_id in range(1, 11):
#     # prepare cross-validation data
#     X_te = X_train1[training51.index[training51["fold"] == fold_id]]
#     X_tr = X_train1[training51.index[training51["fold"] != fold_id]]
#     Y_te = Y_train[training51.index[training51["fold"] == fold_id]]
#     Y_tr = Y_train[training51.index[training51["fold"] != fold_id]]

#     base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
#     base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

#     # build model & run MCMC
#     bma_prior, bma_gp_config = bma_dist(X_tr, 
#                                     base_preds_tr, 
#                                     **bma_model_config)

#     bma_model_config.update(bma_gp_config)

#     print("BMA model config: ", bma_model_config, "mcmc_config: ", mcmc_config, "map_config: ", map_config)
#     bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
#                                            model_config=bma_model_config,
#                                            Y=Y_tr, 
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config)


#     bma_joint_samples = make_bma_samples(X_te, None, base_preds_te, 
#                                      bma_weight_samples=bma_gp_w_samples[0],
#                                      bma_model_config=bma_model_config,
#                                      n_samples=bma_n_samples_eval, 
#                                      seed=bne_seed,
#                                      y_samples_only=False)

#     y_pred = bma_joint_samples['y']
#     y_pred = tf.reduce_mean(y_pred, axis=0)
#     rmse_bma.append(rmse(Y_te, y_pred))
#     print(rmse_bma)

#     # Construct data from BMA samples, shapes (num_samples * num_data, ...)
#     means_tr_mcmc, X_tr_mcmc, Y_tr_mcmc = make_bma_samples(
#         X_tr, Y_tr, base_preds_tr, 
#         bma_weight_samples=bma_gp_w_samples[0],
#         bma_model_config=bma_model_config,
#         n_samples=bma_n_samples_train,
#         seed=bma_seed, 
#         prepare_mcmc_training=True)

#     # Mean samples based on test data, shape (num_samples, num_data, num_output).
#     # It is used to generate final examples in `make_bne_samples()`.
#     means_te_mcmc = make_bma_samples(
#         X_te, None, base_preds_te, 
#         bma_weight_samples=bma_gp_w_samples[0],
#         bma_model_config=bma_model_config,
#         n_samples=bma_n_samples_test,
#         seed=bma_seed)

#     # # # BNE GP Configs.
# # # lengthscale = 1. # @param
# # # l2_regularizer = 10. # @param

# # BNE model configs. 
# # If estimate_mean=False, only estimates a constant variance on top of the 
# # original model.
#     estimate_mean = "True" # @param ["True", "False"]
#     variance_prior_mean=0. # @param
# # # MAP and MCMC configs
# # map_step_size=0.1 # @param
# # map_num_steps=10_000 # @param

# # mcmc_step_size=1e-2 # @param
# # mcmc_num_steps=10_000 # @param

#     bne_gp_config = DEFAULT_GP_CONFIG.copy()
#     bne_model_config = DEFAULT_BNE_CONFIG.copy()

#     map_config = DEFAULT_MAP_CONFIG.copy()
#     mcmc_config = DEFAULT_MCMC_CONFIG.copy()


#     bne_gp_config.update(dict(lengthscale=bne_gp_lengthscale, 
#                           l2_regularizer=bne_gp_l2_regularizer))
#     bne_model_config.update(dict(estimate_mean=eval(estimate_mean),
#                              variance_prior_mean=variance_prior_mean,
#                              **bne_gp_config))

#     map_config.update(dict(learning_rate=map_step_size,
#                        num_steps=map_num_steps))
#     mcmc_config.update(dict(step_size=mcmc_step_size, 
#                         num_steps=mcmc_num_steps,
#                        burnin=mcmc_burnin,
#                        nchain=mcmc_nchain))
    
#     # build model & run MCMC
#     bne_prior, bne_gp_config = bne_model_dist(
#             inputs=X_tr_mcmc,
#             mean_preds=means_tr_mcmc,
#             **bne_model_config)

#     bne_model_config.update(bne_gp_config)

#     print("BNE model config: ", bne_model_config, "mcmc_config: ", mcmc_config, "map_config: ", map_config)
#     # Estimates GP weight posterior using MCMC.
#     bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
#                                             model_config=bne_gp_config,
#                                             Y=Y_tr_mcmc,
#                                             map_config=map_config,
#                                             mcmc_config=mcmc_config,
#                                             initialize_from_map=True)
#     # Generates the posterior sample for all model parameters. 
#     bne_joint_samples = make_bne_samples(X_te,
#                                         mean_preds=means_te_mcmc,
#                                         bne_model_config=bne_model_config,
#                                         bne_weight_samples=bne_gp_w_samples[0],
#                                         seed=bne_seed)
    
#     y_pred = bne_joint_samples['y']
#     means_pred = np.mean(y_pred, axis=0)
#     rmse_bne.append(rmse(Y_te, y_pred))
#     print(rmse_bne)

    
# print(rmse_bma, rmse_bne)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


BMA model config:  {'lengthscale': 0.05, 'l2_regularizer': 0.1, 'hidden_units': 1024, 'y_noise_std': 0.01, 'units': 3, 'seed': 0} mcmc_config:  {'step_size': 0.0001, 'num_steps': 1000, 'burnin': 2500, 'nchain': 10, 'seed': 0} map_config:  {'learning_rate': 0.0005, 'num_steps': 10000}
Running MAP:	296986.125...114104.2421875...105060.953125...101405.3984375...99391.765625...98193.171875...97492.0234375...97085.65625...96843.9375...96694.2109375...Done.
Running MCMC:	Acceptance Ratio: 0.6797609925270081
[1.7436911]
BNE model config:  {'estimate_mean': True, 'estimate_variance': False, 'estimate_skewness': False, 'variance_prior_mean': 0.0, 'skewness_prior_mean': 0.0, 'lengthscale': 0.05, 'l2_regularizer': 1.0, 'hidden_units': 1024, 'y_noise_std': -1.0, 'units': 2, 'seed': 0} mcmc_config:  {'step_size': 0.0001, 'num_steps': 1000, 'burnin': 2500, 'nchain': 10, 'seed': 0} map_config:  {'learning_rate': 0.0005, 'num_steps': 10000}
Running MAP:	8963.25390625...6418.03564453125...6255.54052734

KeyboardInterrupt: 

### Build model

In [9]:
bma_prior, bma_gp_config = bma_dist(X_train1, 
                                    base_preds_train, 
                                    **bma_model_config)

bma_model_config.update(bma_gp_config)

# Check if the model graph is specified correctly.
bma_prior.resolve_graph()

/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


(('gp_weights', ()), ('y', ('gp_weights',)))

In [10]:
display(bma_model_config)

{'lengthscale': 0.1,
 'l2_regularizer': 0.5,
 'hidden_units': 1024,
 'y_noise_std': 0.01,
 'activation': 'relu',
 'units': 3,
 'seed': 0}

### Run MCMC

In [11]:
# bma_gp_w_samples, chain_samples, sampler_stat = run_posterior_inference(model_dist=bma_prior, 
#                                            model_config=bma_model_config,
#                                            Y=Y_train, 
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config)

# Above the debug mode

bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_train, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


bma_joint_samples = make_bma_samples(X_test1, None, base_preds_test, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config,
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)



Running MAP:	397364.5...146621.109375...131055.984375...126781.3671875...123278.0390625...121348.90625...120506.4765625...120074.546875...119810.96875...119624.8828125...Done.
Running MCMC:	

### Prepare Data for BAE/BNE

In [15]:
# # Construct data from BMA samples, shapes (num_samples * num_data, ...)
# means_train_mcmc, X_train_mcmc, Y_train_mcmc = make_bma_samples(
#     X_train1, Y_train, base_preds_train, 
#     bma_weight_samples=bma_gp_w_samples[0],
#     bma_model_config=bma_model_config,
#     n_samples=bma_n_samples_train,
#     seed=bma_seed, 
#     prepare_mcmc_training=True)

# # Mean samples based on test data, shape (num_samples, num_data, num_output).
# # It is used to generate final examples in `make_bne_samples()`.
# means_test_mcmc = make_bma_samples(
#     X_test1, None, base_preds_test, 
#     bma_weight_samples=bma_gp_w_samples[0],
#     bma_model_config=bma_model_config,
#     n_samples=bma_n_samples_test,
#     seed=bma_seed)

/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


## Bayesian Additive Ensemble

Given $\mu(x)$ the posterior of a Bayesian ensemble model, the Bayesian Additive Ensemble is defined as:    

$y \sim N(\mu(x) + r(x), \sigma^2)$

$r \sim GaussianProcess(0, k)$

The additive ensemble $r(x)$ services two purposes: 

1. Mitigates systematic bias in model prediction; 
2. Quantifies the model's epistemic uncertainty.

In [17]:
# # # # BNE GP Configs.
# # # lengthscale = 1. # @param
# # # l2_regularizer = 10. # @param

# # BNE model configs. 
# # If estimate_mean=False, only estimates a constant variance on top of the 
# # original model.
# estimate_mean = "True" # @param ["True", "False"]
# variance_prior_mean=0. # @param
# # # MAP and MCMC configs
# # map_step_size=0.1 # @param
# # map_num_steps=10_000 # @param

# # mcmc_step_size=1e-2 # @param
# # mcmc_num_steps=10_000 # @param

# bne_gp_config = DEFAULT_GP_CONFIG.copy()
# bne_model_config = DEFAULT_BNE_CONFIG.copy()

# map_config = DEFAULT_MAP_CONFIG.copy()
# mcmc_config = DEFAULT_MCMC_CONFIG.copy()


# bne_gp_config.update(dict(lengthscale=bne_gp_lengthscale, 
#                           l2_regularizer=bne_gp_l2_regularizer))
# bne_model_config.update(dict(estimate_mean=eval(estimate_mean),
#                              variance_prior_mean=variance_prior_mean,
#                              **bne_gp_config))

# map_config.update(dict(learning_rate=map_step_size,
#                        num_steps=map_num_steps))
# mcmc_config.update(dict(step_size=mcmc_step_size, 
#                         num_steps=mcmc_num_steps,
#                        burnin=mcmc_burnin,
#                        nchain=mcmc_nchain))

In [18]:
# spcv_df = tf.repeat(training51["fold"], bma_n_samples_train, axis=0)
# rmse_bae = []
# for fold_id in range(1,11):
#     X_te = X_train_mcmc.numpy()[spcv_df==fold_id]
#     X_tr = X_train_mcmc.numpy()[spcv_df!=fold_id]
#     Y_te = Y_train_mcmc[spcv_df==fold_id]
#     Y_tr = Y_train_mcmc[spcv_df!=fold_id]
#     means_tr_mcmc = means_train_mcmc.numpy()[spcv_df!=fold_id]
#     means_te_mcmc = means_train_mcmc.numpy()[spcv_df==fold_id]

#     base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
#     base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

#     # Construct posterior sampler.
#     bne_prior, bne_gp_config = bne_model_dist(
#         inputs=X_tr,
#         mean_preds=means_tr_mcmc,
#         **bne_model_config)

#     bne_model_config.update(bne_gp_config)


#     # Estimates GP weight posterior using MCMC.
#     bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
#                                             model_config=bne_gp_config,
#                                             Y=Y_tr,
#                                             map_config=map_config,
#                                             mcmc_config=mcmc_config,
#                                             initialize_from_map=True)
#     # Generates the posterior sample for all model parameters. 
#     bne_joint_samples = make_bne_samples(X_te,
#                                         mean_preds=means_te_mcmc,
#                                         bne_model_config=bne_model_config,
#                                         bne_weight_samples=bne_gp_w_samples[0],
#                                         seed=bne_seed)
    
#     y_pred = bne_joint_samples['y']
#     means_pred = np.mean(y_pred, axis=0)
#     rmse_bae.append(np.sqrt(np.mean((means_pred - Y_te)**2)))
# print("10 fold cross validation rmse:", rmse_bae)
# print("mean rmse:", np.mean(rmse_bae))


Running MAP:	9345.3330078125...6812.9853515625...6520.54736328125...6457.919921875...6431.615234375...6418.30615234375...6410.1630859375...6405.15673828125...6402.2373046875...6400.5810546875...Done.
Running MCMC:	Acceptance Ratio: 0.7659201622009277
Running MAP:	11660.6328125...8991.4853515625...8634.99609375...8541.29296875...8500.9951171875...8478.3271484375...8464.1328125...8455.822265625...8451.666015625...8450.060546875...Done.
Running MCMC:	Acceptance Ratio: 0.7294092774391174
Running MAP:	12507.0283203125...8742.0400390625...8424.658203125...8292.90234375...8211.7197265625...8147.400390625...8095.224609375...8053.931640625...8022.33349609375...7998.8974609375...Done.
Running MCMC:	Acceptance Ratio: 0.7653064727783203
Running MAP:	12239.8583984375...8483.6669921875...8200.3193359375...8101.00830078125...8036.7373046875...7986.86181640625...7948.244140625...7919.75341796875...7899.66845703125...7885.6552734375...Done.
Running MCMC:	Acceptance Ratio: 0.8057963252067566
Running MAP

### Build Model

In [ ]:
# # Construct posterior sampler.
# bne_prior, bne_gp_config = bne_model_dist(
#     inputs=X_train_mcmc,
#     mean_preds=means_train_mcmc,
#     **bne_model_config)

# bne_model_config.update(bne_gp_config)
# print(f'prior model graph: {bne_prior.resolve_graph()}')

prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))


### Run MCMC

In [ ]:
# # Estimates GP weight posterior using MCMC.
# bne_gp_w_samples, chain_samples, sampler_stat = run_posterior_inference(model_dist=bne_prior,
#                                            model_config=bne_gp_config,
#                                            Y=Y_train_mcmc,
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config,
#                                            initialize_from_map=True)
# # Generates the posterior sample for all model parameters. 
# bne_joint_samples = make_bne_samples(X_test1,
#                                      mean_preds=means_test_mcmc,
#                                      bne_model_config=bne_model_config,
#                                      bne_weight_samples=bne_gp_w_samples[0],
#                                      seed=bne_seed)

Running MAP:	8040.71630859375...7257.259765625...7244.951171875...7239.53125...7238.07568359375...7237.91650390625...7237.91259765625...7237.912109375...7237.912109375...7237.912109375...Done.
Running MCMC:	Acceptance Ratio: 0.7726417183876038


In [ ]:
# surface_pred_bae = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
# surface_var_bae = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}

# # dealing with NA

## Bayesian Nonparametric Ensemble (Variance Only)
So far, we are only estimating the mean-component of the model, i.e., we are assuming: 

$y \sim Gaussian(m(x), \sigma^2); \quad m(x) = GP(0, k)$.

By doing so, the model is implicitly assuming the distribution of $y$ is always a symmetric Gaussian distribution with constant mean across space and time. As a result, our model can only quantify model uncertainty (due to lack of data) via the GP prior, but cannot flexibly capture the data uncertainty that is inherent to the empirical distribution of y.

To resolve this, we extend the ensemble's outcome distribution $y | f$ by also estimating the higher moments of the data distribution (e.g., variance, skewness, etc) using flexible estimators. Specifically, we specify the outcome distribution family to the [maximum-entropy distribution](https://en.wikipedia.org/wiki/Principle_of_maximum_entropy) given the known moments, so the predictive distribution is [minimax](https://arxiv.org/pdf/math/0410076.pdf) and still statistically efficient to estimate.

For example, when we want to estimate the first two moments (mean and variance) of the distribution, this leads to a Gaussian distribution with spatio-temporally adaptive variance $\sigma(x)^2$:

$$y \sim Gaussian(m(x), \sigma(x)^2); \quad \mbox{where} \quad m \sim GP(0, k_m), \sigma \sim GP(0, k_\sigma)$$

and when we want to estimate the first three moments (mean and variance) of the distribution, this leads to a [Exponentially-modifed Gaussian](https://en.wikipedia.org/wiki/Exponentially_modified_Gaussian_distribution) (EMG) distribution with spatio-temporally adaptive variance $\sigma(x)^2$ and skewness $\lambda(x)$:

$$y \sim EMG(m(x), \sigma(x)^2, \lambda(x)); \quad \mbox{where} \quad m \sim GP(0, k_m), \sigma \sim GP(0, k_\sigma), \lambda \sim GP(0, k_\lambda)$$




### Define Model & Run MCMC

In [53]:
# # Construct posterior sampler.
# bne_prior, bne_gp_config = bne_model_dist(
#     inputs=X_train_mcmc,
#     mean_preds=means_train_mcmc,
#     **bne_model_config)

# bne_model_config.update(bne_gp_config)
# print(f'prior model graph: {bne_prior.resolve_graph()}')

# # Estimates GP weight posterior using MCMC.
# bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
#                                            model_config=bne_gp_config,
#                                            Y=Y_train_mcmc,
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config,
#                                            initialize_from_map=True)
# # Generates the posterior sample for all model parameters. 
# bne_joint_samples = make_bne_samples(X_test1,
#                                      mean_preds=means_test_mcmc,
#                                      bne_model_config=bne_model_config,
#                                      bne_weight_samples=bne_gp_w_samples[0],
#                                      seed=bne_seed)

/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))
Running MAP:	12865.1201171875...9376.412109375...9139.1455078125...9043.2568359375...8973.501953125...8915.640625...8868.724609375...8832.21875...8804.4267578125...8783.3193359375...Done.
Running MCMC:	Acceptance Ratio: 0.7179597616195679


In [ ]:
# surface_pred_bne_vo = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
# surface_var_bne_vo = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}



## Bayesian Nonparametric Ensemble (Variance + Skewness)

### Define Model & Run MCMC

In [ ]:
# # Construct prior distribution.
# bne_prior, bne_gp_config = bne_model_dist(
#     inputs=X_train_mcmc,
#     mean_preds=means_train_mcmc,
#     **bne_model_config)

# bne_model_config.update(bne_gp_config)
# print(f'prior model graph: {bne_prior.resolve_graph()}')
# # Estimates GP weight posterior using MCMC.
# bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
#                                            model_config=bne_gp_config,
#                                            Y=Y_train_mcmc,
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config,
#                                            initialize_from_map=True)
# # Generates the posterior sample for all model parameters. 
# bne_joint_samples = make_bne_samples(X_test1,
#                                      mean_preds=means_test_mcmc,
#                                      bne_model_config=bne_model_config,
#                                      bne_weight_samples=bne_gp_w_samples[0],
#                                      seed=bne_seed)

prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))
Running MAP:	7174.9990234375...7055.798828125...7053.3818359375...7053.35888671875...7053.3583984375...7053.35888671875...7053.35791015625...7053.35791015625...7053.35791015625...7053.3583984375...Done.
Running MCMC:	Acceptance Ratio: 0.7549411058425903


In [ ]:
# surface_pred_bne_vs = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
# surface_var_bne_vs = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}

## Basic Plots

In [ ]:
#_DATA_ADDR_PREFIX = "./example/data"
# Tuning Parameters
BMA_lenthscale = bma_gp_lengthscale
#BNE_lenthscale = bne_gp_lengthscale
BMA_L2 = bma_gp_l2_regularizer
#BNE_L2 = bne_gp_l2_regularizer
_SAVE_ADDR_PREFIX = "./pic/BMA_lenthscale_{}_L2_{}".format(BMA_lenthscale, BMA_L2)

path=_SAVE_ADDR_PREFIX
isExists=os.path.exists(path) #判断路径是否存在，存在则返回true

if not isExists:
    os.makedirs(path)


### 1. The predictive surface of individual base models.

In [53]:
coordinate = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
monitors = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
base_model_names = ["pred_av", "pred_gs", "pred_caces"]

base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]] = np.where(np.isnan(base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]]), 0, base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]])
color_norm_base = make_color_norm(
    base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]],   
    method="percentile")

for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_model_{}_bmals_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer))
    
    posterior_heatmap_2d(base_model_predictions_eastMA[base_model_name], coordinate,
                         monitors,
                         cmap='RdYlGn_r',
                         norm=color_norm_base, 
                         #norm_method="percentile",
                         save_addr=save_name)

### 2. The predictive surface of individual BNE gp weights

In [ ]:
bma_ensemble_weights = bma_joint_samples['ensemble_weights']
ensemble_weights_val = tf.reduce_mean(bma_ensemble_weights, axis=0)

weights_dict = {
    "AV": ensemble_weights_val[:, 0],
    "GS": ensemble_weights_val[:,1],
    "CACES": ensemble_weights_val[:,2],
}
#weights_dict
color_norm_weights = make_color_norm(
    list(weights_dict.values()),#[2],   
    method="percentile")

In [ ]:
ensemble_weights_var = np.var(bma_ensemble_weights, axis=0)
weights_var_dict = {
    "AV": ensemble_weights_var[:, 0],
    "GS": ensemble_weights_var[:,1],
    "CACES": ensemble_weights_var[:,2],
}
#weights_dict
color_norm_weights_var = make_color_norm(
    list(weights_var_dict.values()),#[0],   
    method="percentile")
# display(ensemble_weights_val,ensemble_weights_var)

In [ ]:
base_model_names = ["AV", "GS", "CACES"]
for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_weights_{}_bmals_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer))
    
    posterior_heatmap_2d(weights_dict[base_model_name], coordinate,
                         monitors,
                         cmap='viridis',
                         norm=color_norm_weights, 
                         #norm_method="percentile",
                         #save_addr='')
                         save_addr=save_name)

In [ ]:
# plot weights' variance
for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_wvar_{}_bmals_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer))
    
    posterior_heatmap_2d(weights_var_dict[base_model_name], coordinate,
                         monitors,
                         cmap='viridis',
                         norm=color_norm_weights_var, 
                         #norm_method="percentile",
                         #save_addr='')
                         save_addr=save_name)

### 3. The predictive surface of Y_mean, residual process, and Y_mean + residual process.

In [ ]:
# # BAE
# color_norm_pred = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
#     list(surface_pred_bae.values())[:2],  
#     method="percentile")

# color_norm_pred_r = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[2:]),
#     list(surface_pred_bae.values())[2],  
#     method="residual_percentile")


# for name, value in surface_pred_bae.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'BAE_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
#                                 name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
#                                 bne_gp_lengthscale, bne_gp_l2_regularizer))

#     value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                                   cmap='RdYlGn_r',
#                     norm= color_norm_pred_r if name=='resid' else color_norm_pred,
#                     #norm_method="percentile",
#                     #save_addr='')
#                     save_addr=save_name)
    

In [ ]:
# # BNE vo
# color_norm_pred = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
#     list(surface_pred_bne_vo.values())[:2],  
#     method="percentile")

# color_norm_pred_r = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[2:]),
#     list(surface_pred_bne_vo.values())[2],  
#     method="residual_percentile")


# for name, value in surface_pred_bne_vo.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'BNEvo_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
#                                 name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
#                                 bne_gp_lengthscale, bne_gp_l2_regularizer))

#     value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                                   cmap='RdYlGn_r',
#                     norm= color_norm_pred_r if name=='resid' else color_norm_pred,
#                                       save_addr=save_name)
#                     #norm_method="percentile",
#                     #save_addr='')
    

In [ ]:
# # BNE v+s
# color_norm_pred = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
#     list(surface_pred_bne_vs.values())[:2],  
#     method="percentile")

# color_norm_pred_r = make_color_norm(
#     #np.nan_to_num(list(surface_pred_bae.values())[2:]),
#     list(surface_pred_bne_vs.values())[2],  
#     method="residual_percentile")


# for name, value in surface_pred_bne_vs.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'BNEvs_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(name, bma_gp_lengthscale, 
#                                 bma_gp_l2_regularizer, bne_gp_lengthscale, bne_gp_l2_regularizer))

#     value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                                   cmap='RdYlGn_r',
#                     norm= color_norm_pred_r if name=='resid' else color_norm_pred,
#                     #norm_method="percentile",
#                     #save_addr='')
#                 save_addr=save_name)
    

### 4.The predictive variance of Y_mean, residual process, and Y.

In [ ]:
# # BAE
# color_norm_var = make_color_norm(
#     list(surface_var_bae.values())[:2], 
#     method="percentile")

# color_norm_var_r = make_color_norm(
#     list(surface_var_bae.values())[2], 
#     method="percentile")


# for name, value in surface_var_bae.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'var_BAE_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
#                                 name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
#                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
#     #value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                 cmap='inferno_r',
#                                 norm= color_norm_var_r if name=='resid' else color_norm_var,
#                                 #norm_method="percentile",
#                                 save_addr=save_name)

In [ ]:
# # BNE vo
# color_norm_var = make_color_norm(
#     list(surface_var_bne_vo.values())[:2], 
#     method="percentile")

# color_norm_var_r = make_color_norm(
#     list(surface_var_bne_vo.values())[2], 
#     method="percentile")


# for name, value in surface_var_bne_vo.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'var_BNEvo_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
#                                 name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
#                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
#     #value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                 cmap='inferno_r',
#                                 norm= color_norm_var_r if name=='resid' else color_norm_var,
#                                 #norm_method="percentile",
#                                 save_addr=save_name)

In [ ]:
# # BNE v+s
# color_norm_var = make_color_norm(
#     list(surface_var_bne_vs.values())[:2], 
#     method="percentile")

# color_norm_var_r = make_color_norm(
#     list(surface_var_bne_vs.values())[2], 
#     method="percentile")


# for name, value in surface_var_bne_vs.items():
#     save_name = os.path.join(_SAVE_ADDR_PREFIX,
#                             'var_BNEvs_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
#                                 name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
#                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
#     #value = np.where(np.isnan(value), 0, value)
#     color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
#                                 cmap='inferno_r',
#                                 norm= color_norm_var_r if name=='resid' else color_norm_var,
#                                 #norm_method="percentile",
#                                 save_addr=save_name)